 **Note:** Igonre or comment jovian lines if you are running this notebook.

In [ ]:
# The original version of this notebook was downloaded from: 
# https://www.kaggle.com/code/osamaeldemerdash/image-classification-cnn and
# https://www.kaggle.com/code/pranjalsoni17/natural-scene-classification

#project name
project_name = 'tuttle-twins-image-classification'

In [ ]:
!python3 -m pip install --upgrade pip

%pip install pandas
%pip install numpy
%pip install torch
%pip install torchvision
%pip install matplotlib
%pip install opencv-python
print("done")

In [ ]:
#import necessory libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
import matplotlib.pyplot as plt
import cv2
import datetime
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline
print("done")

> # **Exploring the Dataset**

In [ ]:
# e.g. TT_S01_E02_FRM-00-19-05-09.jpg
SOURCE_IMAGES_DIR = "../src-images/"

# pred_data.csv   test_data.csv   train_data.csv
CSV_DATA_DIR = "../csv-data/"

STAGES = ["train", "test", "pred"]

## Pre-process csv data files

In [ ]:
def pre_process_csv_stage_data(stage):
    '''
    read the csv_data file for the given stage
    and return its X_images and y_labels
    '''
    
    assert stage in STAGES
    
    csv_data_file = f"{CSV_DATA_DIR}{stage}_data.csv"
    print(f"pre-processing: {csv_data_file}")
    
    df = pd.read_csv(csv_data_file, header=None, names=['file_name', 'label'])
    X_file_names = df["file_name"].to_list()
    y_labels = df["label"].to_list()
    
    print(f"X_{stage}_file_names: {len(X_file_names)}")
    print(f"y_{stage}labels: {len(y_labels)}")

    # review X image sizes
    X_size = []
    for i in range(len(X_file_names)):
        image = plt.imread(SOURCE_IMAGES_DIR + X_file_names[i])
        X_size.append(image.shape)
    pd.Series(X_size).value_counts()
    
    # Read and resize X_images
    size = 100
    X_images = []
    for i in range(len(X_file_names)):
        im_cv = cv2.imread(SOURCE_IMAGES_DIR + X_file_names[i])
        im_rgb = cv2.cvtColor(im_cv, cv2.COLOR_BGR2RGB)
        image_array = cv2.resize(im_rgb, (size,size), interpolation=cv2.INTER_AREA)
        X_images.append(list(image_array))

    print(f"X_{stage}_images: {len(X_images)} images resized to {size}x{size} pixels")
    
    return (X_images, y_labels)


def create_random_seed(num_digits=3):
    '''
    Use the last num_digits [1..6]chars of the current utc time in 
    ISO8601 format to create a random num_digit number
    '''
    num_digits = min(max(1,num_digits),6)
    iso_str = datetime.datetime.utcnow().isoformat()
    return int(iso_str[-num_digits:])


def display_X_stage_images(stage, X_images, y_labels):
    '''
    Display a random selection of 36 X_images and their y_labels
    '''
    np.random.seed(create_random_seed())
    num_images = min(36, len(X_images))
    print(f"Displaying a random selection of {num_images} X_{stage}_images and their y_{stage}_labels")
    
    plt.figure(figsize=(20,20))
    for n , i in enumerate(list(np.random.randint(0,len(X_images),num_images))) : 
        plt.subplot(6,6,n+1)
        plt.imshow(X_images[i])   
        plt.axis('off')
        plt.title(f"{y_labels[i]} @ {i}")


## Pre-process and display the train data

In [ ]:
(X_train, y_train) = pre_process_csv_stage_data(stage="train")
display_X_stage_images(stage="train", X_images=X_train, y_labels=y_train)


## Pre-process and display the test data

In [ ]:
(X_test, y_test) = pre_process_csv_stage_data(stage="test")
display_X_stage_images(stage="test", X_images=X_test, y_labels=y_test)

## Pre-process and display the pred data


In [ ]:
(X_pred, y_pred) = pre_process_csv_stage_data(stage="pred")
display_X_stage_images(stage="pred", X_images=X_pred, y_labels=y_pred)

> # Training and Validation Datasets : 

In [ ]:
random_seed = 2021
torch.manual_seed(random_seed)

> # Load the datasets into batches:

In [ ]:
# Create train and test datasets
train_ds = tuple(zip(X_train,y_train))
test_ds = tuple(zip(X_test,y_test))

batch_size = 100

# load the datasets into batches
train_dl = DataLoader(
    train_ds, batch_size=batch_size, shuffle=False, 
    num_workers=0, collate_fn=None, pin_memory=False)

test_dl = DataLoader(
    test_ds, batch_size=batch_size*2, shuffle=False, 
    num_workers=0, collate_fn=None, pin_memory=False)

print("train_dl in batches")
for batch_idx, samples in enumerate(train_dl):
      print("***", batch_idx, samples)


In [ ]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig,ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break

> # **Grid Of Train Data Images :**

In [ ]:
# show a 16x8 grid ofimages
show_batch(train_dl)

> # Base Model for Image Classification:

In [ ]:
class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

# Netural Scene Classfication Model:

In [ ]:
class NaturalSceneClassification(ImageClassificationBase):
    
    def __init__(self):
        
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(82944,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,6)
        )
    
    def forward(self, xb):
        return self.network(xb)

In [ ]:
model = NaturalSceneClassification()
model

In [ ]:
for images, labels in train_dl:
    print('images.shape:', images.shape)
    out = model(images)
    print('out.shape:', out.shape)
    print('out[0]:', out[0])
    break

Helper Function or classes to Load Data into GPU

In [ ]:
def get_default_device():
    """ Set Device to GPU or CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    

def to_device(data, device):
    "Move data to the device"
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking = True)

class DeviceDataLoader():
    """ Wrap a dataloader to move data to a device """
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        """ Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b,self.device)
            
    def __len__(self):
        """ Number of batches """
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
# load the train and validation data into the device
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device)

> # **Model Fitting**

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [ ]:
#load the model to the device
model = to_device(NaturalSceneClassification(),device)

In [ ]:
#initial evaluation of the model
evaluate(model,val_dl)

In [ ]:
#set the no. of epochs, optimizer funtion and learning rate
num_epochs = 30
opt_func = torch.optim.Adam
lr = 0.001

In [ ]:
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

> # Graphs for Model Accuracy and Losses :

In [ ]:
def plot_accuracies(history):
    """ Plot the history of accuracies"""
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');
    

plot_accuracies(history)

In [ ]:
def plot_losses(history):
    """ Plot the losses in each epoch"""
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

plot_losses(history)

> # Evaluate Test Data :

In [ ]:
# Apply the model on test dataset and Get the results
test_loader = DeviceDataLoader(DataLoader(test_dataset, batch_size*2), device)
result = evaluate(model, test_loader)
result

In [ ]:
#save the model
torch.save(model.state_dict(), 'natural-scene-classification.pth')

> ## Predicting for invisual images:

In [ ]:
def predict_img_class(img,model):
    """ Predict the class of image and Return Predicted Class"""
    img = to_device(img.unsqueeze(0), device)
    prediction =  model(img)
    _, preds = torch.max(prediction, dim = 1)
    return dataset.classes[preds[0].item()]

In [ ]:
from PIL import Image

#open image file
img = Image.open("../input/intel-image-classification/seg_pred/seg_pred/10004.jpg")

#convert image to tensor
img = transforms.ToTensor()(img)

#print image
plt.imshow(img.permute(1,2,0))

#prdict image label
print(f"Predicted Class : {predict_img_class(img,model)}")

In [ ]:
#open image file
img = Image.open("../input/intel-image-classification/seg_pred/seg_pred/10100.jpg")

#convert image to tensor
img = transforms.ToTensor()(img)

#print image
plt.imshow(img.permute(1,2,0))

#prdict image label
print(f"Predicted Class : {predict_img_class(img,model)}")

In [ ]:
#open image file
img = Image.open("../input/intel-image-classification/seg_pred/seg_pred/10241.jpg")

#convert image to tensor
img = transforms.ToTensor()(img)

#print image
plt.imshow(img.permute(1,2,0))

#prdict image label
print(f"Predicted Class : {predict_img_class(img,model)}")

In [ ]:
img.shape

<!-- Save the parmeters to jovian plateform -->